Keras 事前学習モデル

elephant.jpg
<img src="elephant.jpg">

In [1]:
import os
os.getcwd()

'C:\\Users\\hk\\GitHub\\Hello-World'

In [18]:
#Classify ImageNet classes with ResNet50
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
results = decode_predictions(preds, top=5)[0]
for result in results:
    print(result)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
#print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

('n02504458', 'African_elephant', 0.42468706)
('n01871265', 'tusker', 0.31053025)
('n02437312', 'Arabian_camel', 0.12811293)
('n02504013', 'Indian_elephant', 0.02819019)
('n03967562', 'plow', 0.026223024)


In [17]:
#Extract features with VGG16
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

#model = VGG16(weights='imagenet', include_top=False)
model = VGG16(weights='imagenet', include_top=True)

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Top-5のクラスを予測する
# VGG16の1000クラスはdecode_predictions()で文字列に変換される
preds = model.predict(x)
results = decode_predictions(preds, top=5)[0]
for result in results:
    print(result)
#features = model.predict(x)

553467904/553467096 [==============================] - 127s 0us/step
('n02437312', 'Arabian_camel', 0.5636116)
('n01871265', 'tusker', 0.21521364)
('n02504458', 'African_elephant', 0.12651756)
('n01518878', 'ostrich', 0.027002938)
('n02504013', 'Indian_elephant', 0.011259011)


In [19]:
#Extract features from an arbitrary intermediate layer with VGG19
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)

preds = base_model.predict(x)
results = decode_predictions(preds, top=5)[0]
for result in results:
    print(result)

574717952/574710816 [==============================] - 126s 0us/step
('n02504458', 'African_elephant', 0.35015115)
('n01871265', 'tusker', 0.3413904)
('n02437312', 'Arabian_camel', 0.21521164)
('n02504013', 'Indian_elephant', 0.031749684)
('n03743016', 'megalith', 0.021357097)


In [23]:
# keras.applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

model = InceptionV3(weights='imagenet')

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
results = decode_predictions(preds, top=5)[0]
for result in results:
    print(result)

96116736/96112376 [==============================] - 35s 0us/step
('n02504458', 'African_elephant', 0.52061784)
('n01871265', 'tusker', 0.47323295)
('n02504013', 'Indian_elephant', 0.006090448)
('n02113799', 'standard_poodle', 1.878359e-05)
('n02437312', 'Arabian_camel', 5.946949e-06)


In [1]:
#Fine-tune InceptionV3 on a new set of classes
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(...)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)

Using TensorFlow backend.


87916544/87910968 [==============================] - 72s 1us/step


ValueError: `steps_per_epoch=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps_per_epoch` or use the `keras.utils.Sequence` class.

In [2]:
#Build InceptionV3 over a custom input tensor
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_data_format() == 'channels_last'

model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

96116736/96112376 [==============================] - 41s 0us/step


In [1]:
#keras.applications.xception.Xception(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
import numpy as np

model = Xception(weights='imagenet')

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
results = decode_predictions(preds, top=5)[0]
for result in results:
    print(result)

Using TensorFlow backend.


('n02504458', 'African_elephant', 0.71951663)
('n01871265', 'tusker', 0.2616238)
('n02504013', 'Indian_elephant', 0.016356446)
('n02437312', 'Arabian_camel', 0.0005268729)
('n11879895', 'rapeseed', 6.199279e-05)


In [2]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [3]:
print("テンソル入力のリスト(model.inputs)")
print(model.inputs)
print("テンソル出力のリスト(model.outputs)")
print(model.outputs)
print("モデルのグラフで構成される層を平坦化したリスト(model.layers)")
for nn in range(len(model.layers)):
    print("layer -",nn+1, model.layers[nn])

テンソル入力のリスト(model.inputs)
[<tf.Tensor 'input_1:0' shape=(?, ?, ?, 3) dtype=float32>]
テンソル出力のリスト(model.outputs)
[<tf.Tensor 'predictions/Softmax:0' shape=(?, 1000) dtype=float32>]
モデルのグラフで構成される層を平坦化したリスト(model.layers)
layer - 1 <keras.engine.input_layer.InputLayer object at 0x000001920402CA90>
layer - 2 <keras.layers.convolutional.Conv2D object at 0x000001920402E048>
layer - 3 <keras.layers.normalization.BatchNormalization object at 0x000001920402CF98>
layer - 4 <keras.layers.core.Activation object at 0x000001920F0378D0>
layer - 5 <keras.layers.convolutional.Conv2D object at 0x0000019202A5FC50>
layer - 6 <keras.layers.normalization.BatchNormalization object at 0x000001920F076518>
layer - 7 <keras.layers.core.Activation object at 0x000001920F1226D8>
layer - 8 <keras.layers.convolutional.SeparableConv2D object at 0x000001920F2A1DD8>
layer - 9 <keras.layers.normalization.BatchNormalization object at 0x000001920F2E6F28>
layer - 10 <keras.layers.core.Activation object at 0x000001920F373240>
l